In [38]:
import requests
import pandas as pd
from bs4 import BeautifulSoup
import re

In [51]:
%%time

#counter to check the number of house ads scraped
house_no =0

#dicitionary to store housing details
house_details = {}

#searching criteria
regex = re.compile('[ft2]+')
furn_search = re.compile('furnished')
type_search = re.compile('apartment|condo|duplex|flat|house|townhouse|land')
parking_search = re.compile('attached garage|detached garage|street parking|no parking')

#original url
url = 'https://vancouver.craigslist.org/d/real-estate/search/rea'

#looping through each main page
while True:
    response = requests.get(url)
    data = response.text
    soup = BeautifulSoup(data, 'html.parser')
    houses = soup.find_all('p', class_='result-info')
    #looping through each ad in a page
    for house in houses:
        link = house.find('a', class_='result-title hdrlnk').get('href')
        #print(link)
        house_response = requests.get(link)
        house_data = house_response.text
        house_soup = BeautifulSoup(house_data, 'html.parser')
        
        #Getting the price
        price_tag = house_soup.find('span', class_='price')
        price = price_tag.text[1:] if price_tag else 'NA'
        
        #Getting area and bedroom
        housing_tag = house_soup.find('span', class_='housing')

        if housing_tag:
            bedroom = housing_tag.text[2:3]
            #print(len(housing_tag.text))
            #print(housing_tag.text)

            c = housing_tag.text.split('-')
            for i in c:
                i = i.strip()
                #print(i)
                ft = re.compile('[ft2]+')
                start_post = re.compile('\s')
                if ft.search(i):
                        #print(i)
                        end= regex.search(i).span()[0]
                        #print(regex.search(i).span())
                        #print(end)
                        if start_post.search(i):
                            start = start_post.search(i).span()[1]
                            #print(start_post.search(i).span())
                        else:
                            start =0
                        #print(start)
                        #print(i[start:end])
                        area = i[start:end]

            if area is None:
                area = 'NA'
        else:
            area = 'NA'
            bedroom ='NA'

        #Getting title of the ad
        title_tag = house_soup.find('span', id='titletextonly')
        title = title_tag.text if title_tag else 'NA'
        #print(title)
        
        #Getting the neighborhood
        place_tag = house_soup.find('small')
        place = place_tag.text[2:-1] if place_tag else 'NA'

        #Getting the posted date time
        post_datetime_tag = house_soup.find('time', class_='date timeago')
        post_datetime = post_datetime_tag['datetime'] if post_datetime_tag else 'NA'

        divs = house_soup.find('div',class_='mapAndAttrs')
        #print(divs.text)
        #checking if the estate is furnished or unfurnished

        if (furn_search.search(divs.text) is not None):
            finishing = 'furnished'
        else:
            finishing = 'unfurnished'

        #checking the housing type
        if (type_search.search(divs.text) is not None):
            house_type = type_search.search(divs.text).group()
        else:
            house_type = 'NA'

        #checking the parking type
        if (parking_search.search(divs.text) is not None):
            parking_type = parking_search.search(divs.text).group()
        else:
            parking_type = 'NA'

        #checking number of bedroom and bathroom
        data = divs.findNext('p').findNext('span')
        #print(data)
        if ((data.text is not None) and (data.text.find('/')>0)):
            # if there is no value in the bedroom, then check this value
            if bedroom is None:
                bd = data.text.split('/')[0].strip() 
                #print(bd)
                if (bd.find('BR')>0):
                    bedroom = bd[:-2]
                else:
                    bedroom = 'NA'
            #setting the number of bathroom value
            if(data.text.split('/')[1].strip()):
                bth = data.text.split('/')[1].strip()
                #print(bth)
                if (bth.find('Ba')>0):
                    bathroom = bth[:-2]
                else:
                    bathroom = 'NA'

        house_no+=1
    
        #Making an entry for each ad in a dictionary
        house_details[house_no] = [price, bedroom, area, title, place, post_datetime, finishing, house_type, parking_type,
                                   bathroom, link]
     
    #Getting the next page link in the original link
    url_tag = soup.find('a', class_='button next')
    if url_tag.get('href'):
        url = 'https://vancouver.craigslist.org/d/real-estate/search/rea' + url_tag.get('href')
        #print(url)
    else:
        # breaking from the loop in case there is no more link .ie in the last page
        break
    
    #putting a counter to fetch only 1200
    if(house_no==2400):
        break

#Building a data frame from the dictionary created, updated in the above loop
house_df = pd.DataFrame.from_dict(house_details,orient='index', columns=['Price', 'Bedroom', 'Area', 'Title', 'Place', 'Post Timing',
                                                                        'Finishing','House Type', 'Parking Type','Bathroom','Link'])

Wall time: 20min 59s


In [52]:
house_df

,Price,Bedroom,Area,Title,Place,Post Timing,Finishing,House Type,Parking Type,Bathroom,Link
1,2278000,8,6,Beautiful Tsawwassen Home For Sale Custom Buil...,Tsawwassen,2020-07-10T15:42:19-0700,unfurnished,house,attached garage,7,https://vancouver.craigslist.org/van/reb/d/del...
2,684900,NA,NA,AMAZING Assignment Pre-Sale Unit at 567 Clarke...,Burquitlam,2020-07-03T15:44:48-0700,unfurnished,condo,NA,7,https://vancouver.craigslist.org/bnc/reb/d/coq...
3,1098000,4,,Murrayville Basement Entry Home on Private 1/3...,(google map)\n,2020-07-10T15:16:32-0700,unfurnished,house,NA,3,https://vancouver.craigslist.org/rds/reb/d/lan...
4,1975000,4,4100,MAGNIFICENT OCEAN RESIDENCE Westcoast style,SOUTHERN GULF ISLANDS,2020-07-10T14:51:19-0700,unfurnished,house,attached garage,3,https://vancouver.craigslist.org/van/reo/d/pen...
5,1350000,5,11000,5 bedroom home with potential for subdivision ...,port coquitlam,2020-07-10T14:27:14-0700,unfurnished,house,NA,4,https://vancouver.craigslist.org/pml/reo/d/por...
...,...,...,...,...,...,...,...,...,...,...,...
2396,6000000,3,,12 acre farmland,richmond,2020-05-30T23:19:05-0700,unfurnished,land,detached garage,1.5,https://vancouver.craigslist.org/rch/reo/d/ric...
2397,389500,1,554,Fraser Landmark Below Contract Price Pre-saleA...,13955 laurel drive,2020-05-30T22:26:20-0700,unfurnished,apartment,attached garage,1,https://vancouver.craigslist.org/rds/reo/d/sur...
2398,1490000,4,,"Craftsman style home, 6900sf Accessory Bldg, W...","Armstrong, British Columbia",2020-05-30T22:23:06-0700,unfurnished,house,NA,2.5,https://vancouver.craigslist.org/rds/reo/d/blu...
2399,1279888,6,3,►►Just Listed◄◄ Brand New 6 Bed + 5 Bath House...,"Surrey, BC",2020-05-30T22:12:20-0700,unfurnished,house,detached garage,5,https://vancouver.craigslist.org/van/reo/d/sur...


In [54]:
#Saving the dataframe into csv file
house_df.to_csv("Craiglist_HouseData.csv")